In [176]:
import yfinance as yf
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [177]:
#Define a function to create and get the Bollinger Bands
def bollinger_bands(data,window_size = 30):
    rolling_mean = data['Close'].rolling(window=window_size).mean() #Simple Moving Average (SMA)
    rolling_std = data['Close'].rolling(window=window_size).std() #Standard Deviation
    data['UpperBand'] = rolling_mean + (rolling_std * 2) #Upper Bollinger Band
    data['LowerBand'] = rolling_mean - (rolling_std * 2) #Lower Bollinger Band
    return data

In [178]:
# Define a function to compute the Relative Strength Index (RSI)
def RSI(data, window = 13):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    RS = avg_gain / avg_loss
    RSI = 100 - (100 / (1.0 + RS))
    data['RSI'] = RSI
    data['Overbought'] = 70
    data['Oversold'] = 30
    return data
    



In [179]:
# Define a function to create and get the trading strategy
# Buy whem the close price goes below the lower band and the RSI is below 30 and I have no position
# Sell when the close price goes above the upper band and the RSI is above 70 and I have a position

def strategy(data):
    position = 0
    buy_price = []
    sell_price = []
    for i in range(len(data)):
        if data['Close'][i] < data['LowerBand'][i] and data['RSI'][i] < data['Oversold'][i] and position == 0:
            position = 1
            buy_price.append(data['Close'][i])
            sell_price.append(np.nan)
        elif data['Close'][i] > data['UpperBand'][i] and data['RSI'][i] > data['Overbought'][i] and position == 1:
            position = 0
            buy_price.append(np.nan)
            sell_price.append(data['Close'][i])
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
    return (buy_price, sell_price)



In [180]:
# Get the ETH data
data = yf.download('ETH-USD', start='2023-01-10', end='2024-01-18')
# Set the date as index
data.set_index(pd.DatetimeIndex(data.index.values), inplace=True)
# Show the data
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2023-01-10,1321.395508,1342.757202,1318.527222,1336.586060,1336.586060,5830173253
2023-01-11,1336.486816,1387.932739,1323.583130,1387.932739,1387.932739,6314904311
2023-01-12,1405.351440,1432.281250,1378.422119,1417.938477,1417.938477,12230193038
2023-01-13,1417.946167,1461.672729,1404.023926,1451.614868,1451.614868,7684148212
2023-01-14,1451.428467,1563.739136,1450.988403,1550.706909,1550.706909,15444626014


In [181]:
# Add the Bollinger Bands to the data
data = bollinger_bands(data)

In [182]:
# Add the RSI to the data
data = RSI(data)


In [183]:
# Adding a moving average as an additional feature
data['Moving_Average'] = data['Close'].rolling(window=5).mean()

In [184]:
# Preparing the dataset for the machine learning model
features = ['Close', 'UpperBand', 'LowerBand', 'RSI', 'Moving_Average']
data = data.dropna()


In [185]:
# Preparing the dataset for the machine learning model
X = data[features]
y = data['Close'].shift(-1)  # Predicting the next day's close price

# Drop rows where NaN values are present in the shifted target
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.dropna()

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train the RandomForest model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predicting future close prices (handle the NaN values in 'Predicted_Close' appropriately)
data['Predicted_Close'] = np.nan
data.loc[X_test.index, 'Predicted_Close'] = model.predict(X_test)

# Display the data
data.tail()

/var/folders/cp/yd9whbw50ng6k5r1tb2hs4t00000gq/T/ipykernel_56692/3584860486.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Predicted_Close'] = np.nan


,Open,High,Low,Close,Adj Close,Volume,UpperBand,LowerBand,RSI,Overbought,Oversold,Moving_Average,Predicted_Close
2024-01-13,2522.933838,2589.079834,2498.594482,2576.597900,2576.597900,12250316867,2549.463207,2076.330917,197.044933,70,30,2529.521582,2015.147849
2024-01-14,2578.003662,2578.332275,2470.424316,2472.241211,2472.241211,9405587417,2562.084846,2080.569532,425.870532,70,30,2555.004395,2014.860259
2024-01-15,2471.666992,2550.769043,2470.820801,2511.363770,2511.363770,9700630000,2578.486310,2083.130552,351.249843,70,30,2540.856445,2015.147849
2024-01-16,2510.627197,2613.566895,2500.003906,2587.691162,2587.691162,11063317095,2603.182081,2084.515446,165.181404,70,30,2534.470850,2039.592897
2024-01-17,2587.044678,2592.737061,2508.432861,2528.369385,2528.369385,10441017520,2617.458483,2090.978773,217.613849,70,30,2535.252686,NaN


In [186]:
model.score(X_test, y_test)

-1.4756600016428965

In [187]:
# Implement the strategy to the data
buy_price, sell_price = strategy(data)
data['Buy'] = buy_price
data['Sell'] = sell_price


/var/folders/cp/yd9whbw50ng6k5r1tb2hs4t00000gq/T/ipykernel_56692/1406515789.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data['Close'][i] < data['LowerBand'][i] and data['RSI'][i] < data['Oversold'][i] and position == 0:
/var/folders/cp/yd9whbw50ng6k5r1tb2hs4t00000gq/T/ipykernel_56692/1406515789.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif data['Close'][i] > data['UpperBand'][i] and data['RSI'][i] > data['Overbought'][i] and position == 1:
/var/folders/cp/yd9whbw50ng6k5r1tb2hs4t00000gq/T/ipykernel_56692/1406515789.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecate

In [188]:

# Plot the close price, Bollinger bands
plot = data['Close'].hvplot(line_color='blue', label='Close Price', alpha=0.5) * \
       data['UpperBand'].hvplot(line_color='green', label='Upper Band', alpha=0.5) * \
       data['LowerBand'].hvplot(line_color='red', label='Lower Band', alpha=0.5)

# Fill between UpperBand and LowerBand
plot *= data.hvplot.area(x='index', y='LowerBand', y2='UpperBand', alpha=0.3, color='grey')

# Plot Buy and Sell signals
plot *= data.hvplot.scatter(x='index', y='Buy', label='Buy Signal', color='green', marker='^', size=100) * \
        data.hvplot.scatter(x='index', y='Sell', label='Sell Signal', color='red', marker='v', size=100)

# Set options (title, labels)
plot.opts(
    title="Bollinger Bands and RSI Trading Strategy",
    xlabel='Date', ylabel='Close Price USD ($)',
    show_legend=True, width=800, height=400
)

# Display the plot
plot


/Users/DinoK1/anaconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/DinoK1/anaconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .Curve.Close_Price   :Curve   [index]   (Close)
   .Curve.Upper_Band    :Curve   [index]   (UpperBand)
   .Curve.Lower_Band    :Curve   [index]   (LowerBand)
   .Area.I              :Area   [index]   (LowerBand,UpperBand)
   .Scatter.Buy_Signal  :Scatter   [index]   (Buy)
   .Scatter.Sell_Signal :Scatter   [index]   (Sell)

In [189]:
initial_investment = 100000
cash = initial_investment
shares_held = 0
portfolio_values = []
dates = []

# Loop through the data
for date, row in data.iterrows():
    if row['Buy'] and cash > 0:
        # Buy as many shares as possible with available cash
        shares_held = cash / row['Close']
        cash = 0
    elif row['Sell'] and shares_held > 0:
        # Sell all shares held
        cash = shares_held * row['Close']
        shares_held = 0
    # Calculate the current value of the portfolio
    current_value = cash + shares_held * row['Close']
    portfolio_values.append(current_value)
    dates.append(date)

# The final portfolio value and profit or loss
final_value = portfolio_values[-1]
profit_or_loss = final_value - initial_investment

# Create a DataFrame for the portfolio values
portfolio_df = pd.DataFrame({'Date': dates, 'Portfolio Value': portfolio_values})
portfolio_df = portfolio_df.set_index('Date')

# Plotting the portfolio value
portfolio_plot = portfolio_df['Portfolio Value'].hvplot(
    line_color='green', ylabel='Portfolio Value in USD'
)

# Adding a label for the final portfolio value
final_date = dates[-1]
label = hv.Labels({'x': [final_date], 'y': [final_value], 'text': [f"${final_value:,.2f}"]}, ['x', 'y'], 'text')

# Combine the plots
plot = portfolio_plot * label
plot = plot.opts(
    title='Portfolio Value Over Time', 
    xlabel='Date', 
    width=800, height=400,
    tools=['hover'],
     yformatter='%.0f'
)

# Calculating Daily Returns
portfolio_df['Daily Returns'] = portfolio_df['Portfolio Value'].pct_change()

# Calculating Maximum Drawdown
rolling_max = portfolio_df['Portfolio Value'].cummax()
daily_drawdown = portfolio_df['Portfolio Value'] / rolling_max - 1.0
max_drawdown = daily_drawdown.cummin().min()

# Calculating Sharpe Ratio (assuming risk-free rate is 1%)
risk_free_rate = 0.01
sharpe_ratio = (portfolio_df['Daily Returns'].mean() - risk_free_rate) / portfolio_df['Daily Returns'].std() * np.sqrt(252)

# Print the results for Max Drawdown and Sharpe Ratio
print(f"Maximum Drawdown: {max_drawdown * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")


# Print the results
print(f"Final Portfolio Value: ${final_value:,.2f}")
print(f"Profit/Loss: ${profit_or_loss:,.2f}")

# Display the plot
plot



Maximum Drawdown: -18.22%
Sharpe Ratio: -8.72
Final Portfolio Value: $131,050.35
Profit/Loss: $31,050.35


/Users/DinoK1/anaconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/DinoK1/anaconda3/envs/dev/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .Curve.Portfolio_Value :Curve   [Date]   (Portfolio Value)
   .Labels.I              :Labels   [x,y]   (text)